# Feature Engineering
Feature engineering is the art (yes art) of manually creating new features to feed to the model with the goal of boosting performance. In this notebook, I will explore (and save) a number of general and well-known techniques.
This notebook follows two sources:
* Kaggle [course](https://www.kaggle.com/learn/feature-engineering)
* DataCamp [course]()

## Baseline Model
Before proceeding in creating new features, it is recommended to evalute the baseline, with the given initial features. A model performance's improvement reflects the usefulness of the new synthetized features.

## Mututal Information
### Definntion
Having a large number of features might be extremely overwhelming. Thus, the first step is to reduce the available set of features into a smaller one that might serve as a starting point to build a powerful prediction model. Later, more features can be incorporated in the process.
Mutual Information, unlike correlation is capable of capturing any type of relationship between the target and the feature.


### More details
In simple terms, MI reflects the extent to which the knowledge of one variable (feature) reduces the uncertainty of another (target).  
On a technical note, MI is based on information technology measurement refferred to as ***entropy***. The larger the entropy, the more uncertain the variable is, (the less correlated / tied to each other).

### Key Notes
There are important poitns to keep in mind:
* Each feature is evaluated separately. In other words, a feature might quite powerful when combined with other features. Yet, it might not as significant on its own
* A high MI score reflects a **potentially** useful feature. A transformation such as log, exponential, polynomial might be needed to take see the relationship between the target and the feature.
 

### Applying
The kaggle course offers a great opportunity to practice the main usage of this technique. In this subsection, the [automobile 1985](https://www.kaggle.com/datasets/toramky/automobile-dataset) dataset will be used.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [5]:
auto_ds = "Automobile_data.csv"
df = pd.read_csv(auto_ds)